<h2>Описание проекта <a class="tocSkip"> </h2>

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Задачи проекта:**
 - проверить данные выгрузки;
 - сравнить поведение и предпочтения пользователей двух столиц — Москвы и Санкт-Петербурга.
 
**Входные данные** - выгрузка данных с прослушиваниями пользователей Яндекс Музыки (yandex_music_project.csv):
- `userID` — идентификатор пользователя;
- `Track` — название трека;
- `artist` — имя исполнителя;
- `genre` — название жанра;
- `City` — город пользователя;
- `time` — время начала прослушивания;
- `Day` — день недели.

**План работ:**

- [Обзор данных](#start)
- [Предобработка данных](#preprocessing)
- [Проверка гипотез](#hypotheses)
- [Итоги исследования](#end)

<a id="introduction"></a>

<a id="start"></a>

## Обзор данных

In [1]:
#Импортируем библиотеку pandas
import pandas as pd

In [2]:
# Сохраним датасет в переменной df
df = pd.read_csv('yandex_music_project.csv')

In [3]:
#Получим первые 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
#Получим общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В названиях колонок видны нарушения стиля:
* Строчные буквы сочетаются с прописными.
* Встречаются пробелы.
* Не соблюдается стиль написания составных слов



**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

<a id="preprocessing"></a>

[Описание проекта](#introduction)

## Предобработка данных

### Стиль заголовков

In [5]:
#Выведем перечень названий столбцов таблицы df
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


In [6]:
#Переименуем столбцы
df = df.rename(columns = {'  userID': 'user_id',
                          'Track': 'track', '  City  ': 'city',
                         'Day': 'day'})

In [7]:
#Проверим, как изменились названия наших столбцов
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

In [8]:
#Подсчитаем пропуски
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

В нашеим исследовании не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге.
* заполним эти пропуски явными обозначениями;
* оценим, насколько они повредят расчётам. 

In [9]:
#Заменим пропущенные значения на 'unknown'
columns_to_replace = ['track','artist','genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [10]:
#Проверим, остались ли пропуски
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Пропусков не осталось.

### Дубликаты

In [11]:
#Подсчитаем явные дубликаты
df.duplicated().sum()

3826

Всего у нас - 3826 явных дубликатов

In [12]:
#Удалим дубликаты
df = df.drop_duplicates()

In [13]:
#Проверим удаление дубликатов
df.duplicated().sum()

0

In [14]:
#Изучим список уникальных названий жанров для выявления неявных дубликатов
genre_name = df['genre'].sort_values().unique()
print(genre_name)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Были найдены следующие неявные дубликаты:
- `hip`
- `hip-hop`
- `hiphop`
- `hop`

In [15]:
#Устраним найденые дубликаты
duplicates = ['hip','hop','hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)    

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков - исправлены,
- пропущенные значения - заполнены,
- дубликаты — явные и неявные - удалены.

<a id="hypotheses"></a>

[Описание проекта](#introduction)

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

In [16]:
#Подсчитаем количество прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

In [17]:
#Подсчитаем количество прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [18]:
#Напишем функцию, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
#- день недели,
#- название города.

def number_tracks(day, city): #Объявляется функция с двумя параметрами: day, city.

    track_list = df[df['day'] == day] #В переменной track_list сохраняются те строки таблицы df, для которых 
    track_list = track_list[track_list['city'] == city] #значение в столбце 'day' равно параметру day и одновременно значение
                                        #в столбце 'city' равно параметру city
    track_list_count = track_list['user_id'].count() #В переменной track_list_count сохраняется число значений столбца 'user_id'

    return track_list_count #Функция возвращает число - значение track_list_count.


In [19]:
#Количество прослушиваний в Москве по понедельникам
number_tracks('Monday','Moscow')

15740

In [20]:
#Количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday','Saint-Petersburg')

5614

In [21]:
#Количество прослушиваний в Москве по средам
number_tracks('Wednesday','Moscow')

11056

In [22]:
#Количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday','Saint-Petersburg')

7003

In [23]:
#Количество прослушиваний в Москве по пятницам
number_tracks('Friday','Moscow')

15945

In [24]:
#Количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday','Saint-Petersburg')

5895

In [25]:
#Соберем таблицу с полученными результатами
columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow', 15740, 11056, 15945],
       ['Saint-Petersburg', 5614, 7003, 5895]]
info = pd.DataFrame(data=data, columns=columns)
print(info)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу гипотезы о том, что активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [26]:
#Сохраним датасет с данными только по Москве в переменную moscow_general
moscow_general = df[df['city'] == 'Moscow']

In [27]:
#Сохраним датасет с данными только по Питеру в переменную spb_general
spb_general = df[df['city'] == 'Saint-Petersburg']

In [28]:
# Создайте функцию `genre_weekday()` с четырьмя параметрами:
# таблица (датафрейм) с данными,
# день недели,
# начальная временная метка в формате 'hh:mm', 
# последняя временная метка в формате 'hh:mm'.

#Функция будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

def genre_weekday(df, day, time1, time2):
    #Последовательная фильтрация
    #Оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    #Оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2] 
    #Оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    #Сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    #Отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)

    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [29]:
#Результаты функции по Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [30]:
#Результаты функции по Санкт - Петербургу
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [31]:
#Результаты функции по Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [32]:
#Результаты функции по Санкт - Петербургу
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [33]:
#Создадим сводную таблицу с количеством прослушиваний каждого жанра в Москве
moscow_genres = moscow_general.groupby(by = 'genre')['genre'].count().sort_values(ascending = False)

In [34]:
#Выведем первые 10 строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [35]:
#Создадим сводную таблицу с количеством прослушиваний каждого жанра в Санкт - Петербурге
spb_genres = spb_general.groupby(by = 'genre')['genre'].count().sort_values(ascending = False)

In [36]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


<a id="end"></a>

[Описание проекта](#introduction)

## Итоги исследования

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.